In [ ]:
from PIL import Image
import pandas as pd

from nircoloring.config import get_dataset_image_file, DATASET_TEMP_IMAGES
from nircoloring.dataset.caltech import fetch_file_from_blob, load_metadata, load_filenames

In [ ]:
filename = "5a2c8488-23d2-11e8-a6a3-ec086b02610b.jpg"
await fetch_file_from_blob(filename, DATASET_TEMP_IMAGES, scale_and_crop=False)
path = get_dataset_image_file(filename)
img = Image.open(path)

print(img.height)
print(img.width)

img = img.crop((0, 30, img.width, img.height - 90))

crop_size = min(img.width, img.height)
print(crop_size)

left = int((img.width - crop_size) / 2)
top = int((img.height - crop_size) / 2)
right = int((img.width + crop_size) / 2)
bottom = int((img.height + crop_size) / 2)

img = img.crop((left, top, right, bottom))

img.resize((1024, 1024), Image.LANCZOS)

In [ ]:
metadata = load_metadata()

df = pd.DataFrame(data=metadata["images"])
df

In [ ]:
df.groupby(["location"]).count()

In [ ]:
df.groupby(["seq_id"]).count().sort_values(by=["seq_num_frames"],ascending=False)

In [ ]:
filenames = set(load_filenames())
df_ds = pd.DataFrame(data=filter(lambda x: x["file_name"].strip() in filenames, metadata["images"]))
df_ds

In [ ]:
df_ds.groupby(["location"]).size()

In [ ]:
for file_name in df_ds[df_ds.location == "96"]["file_name"][1:6]:
    await fetch_file_from_blob(file_name, DATASET_TEMP_IMAGES)
    image = Image.open(get_dataset_image_file(file_name))
    display(image)